In [1]:
import requests as rq
import pandas as pd

In [2]:
df = pd.read_csv(filepath_or_buffer='catalogo.csv',skiprows = 2, names = ['title', 'activo', 'provider', 'ranking','None','type'])

In [3]:
df['clean_title'] = df['title'].apply(lambda x: x.split('(')[0])

In [4]:
df['clean_title'] = df['clean_title'].apply(lambda x: x.split(':')[0])

In [5]:
df['clean_title'] = df['clean_title'].apply(lambda x: x.split('/')[0])

In [6]:
df['clean_title'] = df['clean_title'].apply(lambda x: x.split('season')[0])

In [7]:
df['clean_title'] = df['clean_title'].apply(lambda x: x.split('Season')[0])

In [8]:
df.shape

(841, 7)

In [9]:
tmdb_url = "https://api.themoviedb.org/3/search/multi?api_key=73380b032ab105b01601564aeebe2f56&language=en-US&page=1&query="

In [10]:
empty_key = {  "backdrop_path":None,
               "first_air_date":None,
               "genre_ids":None,
               "id":None,
               "media_type":None,
               "name":None,
               "origin_country":None,
               "original_language":None,
               "original_name":None,
               "overview":None,
               "popularity":None,
               "poster_path":None,
               "vote_average":None,
               "vote_count":None}

In [11]:
def get_metadata(title):
    resp = rq.get(tmdb_url+title)
    if resp.status_code==200:
        json = resp.json()
        result = json['results'][0] if 'results' in json and len(json['results']) > 0 else empty_key
    else:
        result = empty_key
    return result

In [12]:
meta_data = []
for index, row in df.iterrows():
    title = df["clean_title"][index]
    meta_data.append(get_metadata(title))

In [13]:
len(meta_data)

841

In [14]:
meta_df = pd.DataFrame.from_dict(meta_data)

In [15]:
meta_df.id.count()

667

In [16]:
df.shape

(841, 7)

In [17]:
meta_df.shape

(841, 23)

In [19]:
final_df = pd.concat([df, meta_df],axis=1)

In [20]:
final_df.shape

(841, 30)

In [33]:
final_df[['title','clean_title','name','id']].head(20)

,title,title,clean_title,name,id
0,Andy's Dinosaur Adventures (20 x 15') - Worldwide,NaN,Andy's Dinosaur Adventures,Andy's Dinosaur Adventures,95548.0
1,Andy's Wild Adventures (40 x 15') - Worldwide,NaN,Andy's Wild Adventures,Andy's Wild Adventures,94897.0
2,Fierce Earth: Series 001 (10 x 30') - Broadcast,NaN,Fierce Earth,Fierce Earth,55744.0
3,Gigglebiz: Series 003 (25 x 15') - Broadcast,NaN,Gigglebiz,Gigglebiz,38143.0
4,Horrible Histories: Series 004 (15 x 30') - Edit,NaN,Horrible Histories,Horrible Histories,32419.0
5,Horrible Histories: Series 005 (13 x 30') - Br...,NaN,Horrible Histories,Horrible Histories,32419.0
6,Katie Morag: Series 001 (26 x 15') - Broadcast,NaN,Katie Morag,Katie Morag,122987.0
7,Katie Morag: Series 002 (12 x 15' & 1 x 30') -...,NaN,Katie Morag,Katie Morag,122987.0
8,Kit and Pup (52 x 6') - Edit,NaN,Kit and Pup,None,NaN
9,Mr Bloom's Nursery: Series 003: Get Set Grow! ...,NaN,Mr Bloom's Nursery,Mr Bloom's Nursery,38304.0


In [26]:
final_df.to_csv('catalogo_hidratado.csv')